In [ ]:
!pip install easyocr

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-n

In [ ]:
!pip install pdf2image

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62

Found existing installation: opencv-python-headless 4.5.5.62
Uninstalling opencv-python-headless-4.5.5.62:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.5.5.62.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-64ac49e1.so.58.91.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-4b79e479.so.58.45.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-805734e8.so.56.51.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-018b8c17.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dist

In [ ]:
!pip install opencv-python-headless==4.5.2.52

     |████████████████████████████████| 38.2 MB 1.6 MB/s 


In [ ]:
import easyocr
from pdf2image import convert_from_path
import numpy as np
import requests
from bs4 import BeautifulSoup
import glob
import csv
import json 

In [ ]:
link=[]

with open("Data Engineer Task - Data Engineer Task.csv",'r',encoding="utf8", errors='ignore') as csvfile:
    reader=csv.reader(csvfile,delimiter=',')
    next(reader)
    for row in reader:        
        link.append(row[0])


new_links=[]
for i in link:
    if i[-4:] != '.pdf':     
        response = requests.get(i)

        # parse content
        content = BeautifulSoup(response.text, 'html.parser')

        # extract URLs referencing PDF documents
        all_urls = content.find_all(class_="format-summary download-pill",href=True)
        n=len(all_urls)
        for j  in range(0,n):
          k=all_urls[j]['href']
          if k[-4:]=='.pdf':
            url=('https://archive.org'+k)
            new_links.append(url)
            #print(url)
            break
    else:
      new_links.append(i)  

for link in new_links:        
        with open('link.txt', 'a') as f:
            f.write(link)
            f.write('\n')


#Download the all pdf 
!wget -i link.txt


reader = easyocr.Reader(['mr','hi','en'])

text=[]

pdf_files = glob.glob("*.pdf" )

for file in pdf_files:
    image = convert_from_path(file)
    text = ''
    images=len(image)
    for x in range(images):
      bounds = reader.readtext(np.array(image[x]))
      for y in range(len(bounds)):
        text = text + bounds[y][1] 
        
        
final=[]
for i in range(len(text)):
  valu={
      'page_url':link[i],
      'pdf_url':new_links[i],
      'content':text[i]
  }
  final.append(valu)


 # Python program to write JSON        
# Serializing json
json_object = json.dumps(final, indent = 3)

# Writing to sample.json
with open('output.json', "w") as outfile:
  outfile.write(json_object)